In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession 

In [3]:
spark = SparkSession.builder.appName("Trial").getOrCreate()

22/04/25 19:23:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Download data from [here](https://www.kaggle.com/mczielinski/bitcoin-historical-data)
data = spark.read.csv('bitcoin.csv',header=True,inferSchema=True)

In [5]:
data.printSchema()

root
 |-- Timestamp: integer (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_(BTC): double (nullable = true)
 |-- Volume_(Currency): double (nullable = true)
 |-- Weighted_Price: double (nullable = true)



In [6]:
data.show()

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|1325317980| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318040| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318100| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318160| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318220| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318280| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318340| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318400| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325318460| NaN| NaN| NaN|  NaN|       

### Observation from the data table

There are a lot of NaN values. Check if the compete table is the same way, filled of NaN values or there is some meaningful data. 

### Question 
Confirm the table has more than 1 meaningful occurances. 

In [7]:
# lets arrange the data in accending order of some column say High. if there are values in it they will be first. 
data.orderBy(data.High).show()

[Stage 3:=======================================>                   (2 + 1) / 3]

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1327629480| 3.8| 3.8| 3.8|  3.8|        0.27|            1.026|           3.8|
|1328036700| 3.8| 3.8| 3.8|  3.8|         5.0|             19.0|           3.8|
|1329582120|4.14|4.14|4.14| 4.14|  2.24383721|     9.2894860494|          4.14|
|1329582180|4.14|4.14|4.14| 4.14|  4.58743279|     18.991971751|          4.14|
|1329415380|4.15|4.15|4.15| 4.15|  1.07011258|      4.440967207|          4.15|
|1329358140|4.17|4.17|4.17| 4.17|         4.0|            16.68|          4.17|
|1329601860|4.23|4.23|4.23| 4.23|         5.0|            21.15|          4.23|
|1329567300|4.24|4.24|4.21| 4.21| 11.48636364|     48.454590924|  4.2184447962|
|1329587760|4.25|4.25|4.25| 4.25|        20.0|             85.0|          4.25|
|1329567000|4.27|4.27|4.27| 4.27|       

There are values in the data set it is not completly empty. 

### Question 

Remove all the rows with NaN values.  

In [8]:
dfdrop = data.na.drop()

# drop all rows with empty values 

In [9]:
dfdrop.show()

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|1325346600|4.39|4.39|4.39| 4.39|        48.0|           210.72|          4.39|
|1325350740| 4.5|4.57| 4.5| 4.57| 37.86229723|     171.38033753|  4.5264114983|
|1325350800|4.58|4.58|4.58| 4.58|         9.0|            41.22|          4.58|
|1325391360|4.58|4.58|4.58| 4.58|       1.502|          6.87916|          4.58|
|1325431680|4.84|4.84|4.84| 4.84|        10.0|             48.4|          4.84|
|1325457900| 5.0| 5.0| 5.0|  5.0|        10.1|             50.5|           5.0|
|1325534640| 5.0| 5.0| 5.0|  5.0|      19.048|            95.24|           5.0|
|1325591100|5.32|5.32|5.32| 5.32|  2.41917293|     12.869999988|          5.32|
|1325600520|5.14|5.14|5.14| 5.14|       

### Question 

Find the percentage empty rows.

percentage of non empty rows = (number of non-empty rows/number total number of rows)*100

percentage of empty rows = 100 - percentage of non empty rows

answer = 30.81

In [10]:
100- (  (dfdrop.count()/data.count())*100  )

25.602459928475795

Notice the large amout of time it takes to compute this. It is because .count() is an action it is being performed right now. Moreover, the data set is quite large so it is taking time.

Let's check how large the data set actually is!

In [11]:
dfdrop.count()
#number of rows in dataframe with no empty columns

3613769

In [12]:
data.count()
#number of rows in the complete data set

4857377

### Handeling the timestamp column

Now lets try to convert the timestamp in more readable format.

### Question 
Convert the Timestamp column in normal datetime

hint : you can find the required function here https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.from_unixtime

In [13]:
from pyspark.sql.functions import from_unixtime

In [14]:
df = dfdrop.withColumn('Timestamp',from_unixtime('Timestamp'))

In [15]:
df.show()

+-------------------+----+----+----+-----+------------+-----------------+--------------+
|          Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+-------------------+----+----+----+-----+------------+-----------------+--------------+
|2011-12-31 07:52:00|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|2011-12-31 15:50:00|4.39|4.39|4.39| 4.39|        48.0|           210.72|          4.39|
|2011-12-31 16:59:00| 4.5|4.57| 4.5| 4.57| 37.86229723|     171.38033753|  4.5264114983|
|2011-12-31 17:00:00|4.58|4.58|4.58| 4.58|         9.0|            41.22|          4.58|
|2012-01-01 04:16:00|4.58|4.58|4.58| 4.58|       1.502|          6.87916|          4.58|
|2012-01-01 15:28:00|4.84|4.84|4.84| 4.84|        10.0|             48.4|          4.84|
|2012-01-01 22:45:00| 5.0| 5.0| 5.0|  5.0|        10.1|             50.5|           5.0|
|2012-01-02 20:04:00| 5.0| 5.0| 5.0|  5.0|      19.048|            95.24|           5.0|
|2012-01-03 11:45:00|

### Question 

Find the 90th percentile of weighted price

answer = '8541.8561284'

In [16]:
df.select('Weighted_Price').summary('90%').head(1)[0][1]

'11489.95077'

### QUESTION

Find the date time when the weighted price was maximum

answer = '2017-12-17 12:24:00'

In [17]:
sol1 = df.select(['Timestamp','Weighted_Price']).orderBy(df['Weighted_Price'].desc())

##### Notice:  
How this executes instantly. As there is now action

In [18]:
sol1.head(1)[0][0]

'2021-03-13 20:42:00'

##### Notice:
This takes time because it does the previous transformation first. We are using indexing to get to the timestamp in the firstrow

### Question

Find the total number of occurances when more than 10 Bitcoin was traded

ans = 

In [19]:
df.filter(df['Volume_(BTC)']>10).count()

715191

### Question 

Find the average quantity of bitcoin traded per year 

In [20]:
#import function to find the year form date
#import function to round off average

from pyspark.sql.functions import year
from pyspark.sql.functions import format_number

In [21]:
#Create a new dataframe with a year column

dfyear = df.withColumn('Year',year('Timestamp'))

In [22]:
#find the averages of the yearly quantity

yearavg = dfyear.select(['Year','Volume_(BTC)']).groupBy('Year').mean()

In [23]:
#show the yearly averages in a readable format

yearavg.select('year',format_number('avg(Volume_(BTC))',2).alias("Mean")).show()

+----+-----+
|year| Mean|
+----+-----+
|2018| 7.77|
|2015|14.82|
|2013|15.73|
|2014|12.62|
|2019| 5.89|
|2020| 5.92|
|2012|21.33|
|2016| 5.66|
|2011|23.83|
|2017| 9.72|
|2021| 6.65|
+----+-----+

